In [1]:
import pandas as pd
import imblearn
import numpy as np
import traceback
pd.set_option('display.max_rows', 20,'max_info_columns', 9999,'display.max_columns', 9999)
import matplotlib.pyplot as plt
import copy
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing,metrics
import datetime as dt
import gc
from sklearn import svm,linear_model
from collections import Counter
import collections
from sklearn.model_selection import train_test_split
import csv
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder,Imputer

from sklearn.ensemble import RandomForestRegressor,VotingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedKFold


import jieba #导入结巴分词
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU


from sklearn.feature_selection import RFE 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
import xlrd
warnings.filterwarnings('ignore')

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
train=pd.read_csv('/home/tsl-yu/文档/云移杯/YNU.EDU2018-ScenicWord/train_first.csv')

test=pd.read_csv('/home/tsl-yu/文档/云移杯/YNU.EDU2018-ScenicWord/predict.csv')

In [43]:
sub=pd.read_csv('/home/tsl-yu/文档/云移杯/YNU.EDU2018-ScenicWord_submite_sample.csv',names='a')

In [3]:
y = np_utils.to_categorical(train['Score']-1)

In [4]:
train.drop('Score',axis=1,inplace=True)
train=pd.concat([train,test],axis=0,ignore_index = True)

### 统计詞频 

####  创建停用詞列表，以及句子去掉停用詞

In [5]:
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r',encoding='gb2312').readlines()]
    return stopwords

In [6]:
# 对句子去除停用词
def movestopwords(sentence,stopwords):
    
    outstr = ''
    for word in sentence:
        if word not in stopwords:
            if word != '\t'and'\n':
                outstr += word
                # outstr += " "
    return outstr

In [7]:
stopwl=stopwordslist('/home/tsl-yu/文档/云移杯/Chinese-master/dict/四川大学机器智能实验室停用词库.txt')

In [8]:
stopwl.extend([' ','，','。','的','了',''])

In [9]:

def build_dataset(words, vocabulary_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [10]:
def word_list(df):
    
    w = []
    for i in df:
        w.extend(i)
    return w

## 建立数据集 

In [11]:
cw = lambda x: list(jieba.cut(x,cut_all=True)) #定义分词函数
train['Discuss']=[movestopwords(a,stopwl) for a in train['Discuss']]
train['words'] = train['Discuss'].apply(cw)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.546 seconds.
Prefix dict has been built succesfully.


In [12]:
word=word_list(train['words'])

In [13]:
data, count, dictionary, reverse_dictionary=build_dataset(word,5000)

In [14]:
def get_sent(df,dny):
    out=[]
    for x in df:
        y=[]
        for k in x:
            if k in dny:
                y.append(dny[k])
            else :
                y.append(0)
        out.append(y)
    return out

In [15]:
train['sent']=get_sent(train['words'],dictionary)

In [16]:
maxlen = 10

In [17]:
print("Pad sequences (samples x time)")
train['sent'] = list(sequence.pad_sequences(train['sent'], maxlen=maxlen))


Pad sequences (samples x time)


In [18]:
x = np.array(list(train['sent'][:100000]))[::2] #训练集
y_ = np.array(y)[::2]

xt = np.array(list(train['sent'][:100000]))[1::2] #测试集
yt = np.array(y)[1::2]

x_sub = np.array(list(train['sent'][100000:])) #提交集

In [20]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(dictionary), 256))
model.add(LSTM(256)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x, y_, batch_size=32, nb_epoch=2,validation_data=(xt,yt)) #训练时间为若干个小时



Build model...
Train on 50000 samples, validate on 50000 samples
Epoch 1/2
50000/50000 [==============================] - 62s 1ms/step - loss: 0.9137 - acc: 0.6023 - val_loss: 0.8892 - val_acc: 0.6090
Epoch 2/2
50000/50000 [==============================] - 63s 1ms/step - loss: 0.8522 - acc: 0.6243 - val_loss: 0.8850 - val_acc: 0.6121


In [24]:
out=model.predict_classes(xt,batch_size=32)+1

In [33]:
y_test =np.argmax(yt,axis=1)+1

In [39]:
score=metrics.mean_squared_error(y_test,out)

In [40]:
1/(1+score)

0.5985586707209041

In [46]:
test['Discuss']=model.predict_classes(x_sub,batch_size=32)+1

In [49]:
test.to_csv('sub_first.csv',header=False,index=False)